In [ ]:
def RegridSpectrum(directory, file, R):
    ''' Regrid a spectrum
    
    Args:
        directory (str): location of filder containing orginal spectrum
        file (str): pickle file of dataframe of original spectrum. 
                    Ex: /cloud-free-spectrum-full-output-R2000.pkl
        R (flt): R of new spectrum
    
    '''
    import pickle
    import os
    
    df = pickle.load(open(directory+file,'rb'))
    wno, alb, fpfs, full_output = df['wavenumber'], df['albedo'], df['fpfs_reflected'],  df['full_output']
    if 'cloud-free' in file:
        fileout = 'cloud-free-'
    elif 'cloudy' in file:
        fileout = 'cloudy-'
    
    # Albedo plot
    Rwno, Ralb = jdi.mean_regrid(wno, alb , R = R)
    plt.figure()
    plt.figure(figsize=(8,6))
    plt.minorticks_on()
    plt.tick_params(axis='both',which='major',length =20, width=2,direction='in',labelsize=23)
    plt.tick_params(axis='both',which='minor',length =10, width=2,direction='in',labelsize=23)

    plt.plot(1e4/Rwno, Ralb, lw=2)

    plt.gca().set_xlabel(r'Wavelength [$\mu$m]')
    plt.gca().set_ylabel('Geo Albedo')
    plt.grid(ls=':')
    plt.tight_layout()
    plt.savefig(directory+fileout + 'albedo-spectrum-R' + R + '.png',bbox_inches='tight')

    # reflected plot
    Rwno, Rfpfs = jdi.mean_regrid(wno, fpfs , R = R)
    plt.figure()
    plt.figure(figsize=(8,6))
    plt.minorticks_on()
    plt.tick_params(axis='both',which='major',length =20, width=2,direction='in',labelsize=23)
    plt.tick_params(axis='both',which='minor',length =10, width=2,direction='in',labelsize=23)

    plt.plot(Rwno, Rfpfs, lw=2)
    plt.gca().set_yscale('log')

    plt.gca().set_xlabel(r'Wavelength [$\mu$m]')
    plt.gca().set_ylabel('Planet:Star contrast')
    plt.grid(ls=':')
    plt.tight_layout()
    plt.savefig(directory+fileout + 'fpfs-spectrum-R' + R + '.png',bbox_inches='tight')

    pickle.dump([wno, alb, fpfs, full_output], 
                open(directory+fileout'-spectrum-full-output-R' + R + '.pkl','wb'))
    
    
file = 'ReflectXGasGiantModelGrid-Neptune-Tstar3000-Rstar0.2-Teq187.1302769665348-sep0.1-rad0.35-mass0.05-mh100-co0.5-phase0'
RegridSpectrum(directory, '/cloud-free-spectrum-full-output-R2000.pkl', 150)
